### Record an audio to file

In [1]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pydub
import os
import sounddevice as sd
import soundfile as sf
import numpy as np
from scipy.io.wavfile import write
sr = 16000  # Sample rate
seconds = 3  # Duration of recording
filename = 'myfile.wav'

In [10]:
data = sd.rec(int(seconds * sr), samplerate=sr, channels=1)
sd.wait()  # Wait until recording is finished

In [11]:
data.shape

(48000, 1)

In [16]:
# Convert `data` to 16 bit integers:
y = (np.iinfo(np.int16).max * (data/np.abs(data).max())).astype(np.int16) 
write(filename, sr, y)

<ipython-input-16-b9938c1b9dbd>:2: RuntimeWarning: invalid value encountered in true_divide
  y = (np.iinfo(np.int16).max * (data/np.abs(data).max())).astype(np.int16)


In [17]:
play_file(filename, sr)

In [14]:
# Playback the file
def play_file(filename, sr):
    data, fs = sf.read(filename)  
    sd.play(data, sr)
    status = sd.wait()  # Wait until file is done playing
def play_mp3(filename):
    p = vlc.MediaPlayer(filename)
    p.play()

In [15]:
from pydub import AudioSegment
from pydub.utils import mediainfo
def mp3_to_wav(infile, outfile):
    sr = mediainfo(infile)['sample_rate']
    sound = AudioSegment.from_mp3(infile)
    sound.export(outfile, format="wav")
    return sr

### Get a German transcript from Google ASR

In [ ]:
from google.cloud import speech
import io

In [ ]:
credentials='/home/felix/data/research/Google/AffectiveAgents-7874f11e65d6.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=credentials

In [ ]:
client = speech.SpeechClient()
with io.open(filename, "rb") as audio_file:
    content = audio_file.read()
audio = speech.RecognitionAudio(content = content)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    language_code="de-DE",
)

In [ ]:
response = client.recognize(config=config, audio=audio)
for result in response.results:
    for index, alternative in enumerate(result.alternatives):
        print("Transcript {}: {}".format(index, alternative.transcript))

### Playback synthesized text with Google

In [ ]:
text = 'In {} sind {} Menschen mit {} geimpft.'.format('Berlin', 123, 'Biontech')

In [ ]:
from gtts import gTTS
tts = gTTS(text, lang='de')
audio_file = './out_synth.mp3'
tts.save(audio_file)
sr = mp3_to_wav(audio_file, 'test.wav')
play_file('test.wav', int(sr))

### Playback with pytts (using espeak)

In [18]:
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('voice', 'german')
engine.setProperty('rate', 300)
engine.say(text)
engine.runAndWait()

ModuleNotFoundError: No module named 'pyttsx3'

In [19]:
voices = engine.getProperty('voices') 
for v in voices:
    print(v)

NameError: name 'engine' is not defined

### Voxpopuli (using mbrola)

In [ ]:
from voxpopuli import Voice
voice = Voice(lang="de", pitch=50, speed=200, voice_id=6)
voice.say(text)

In [ ]:
! pip list